In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving archive.zip to archive.zip


In [ ]:
import zipfile
import os

# Extract the ZIP file
zip_path = "archive.zip"  # Update with your uploaded file name if different
extract_path = "./vehicle_dataset"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check extracted contents
os.listdir(extract_path)


['Dataset']

In [ ]:
# Check the folders (classes) inside the dataset
dataset_path = "./vehicle_dataset/Dataset"
classes = os.listdir(dataset_path)
print("Classes:", classes)

# Count images in each class
for cls in classes:
    print(f"{cls}: {len(os.listdir(os.path.join(dataset_path, cls)))} images")


Classes: ['motorcycle', 'Truck', 'Car', 'Bus']
motorcycle: 100 images
Truck: 100 images
Car: 100 images
Bus: 100 images


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.svm import SVC
from skimage.feature import hog
from sklearn.metrics import classification_report


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import glob

# Define parameters
image_size = (128, 128)

# Load images and labels
images = []
labels = []

for label, cls in enumerate(classes):
    cls_folder = os.path.join(dataset_path, cls)
    for img_path in glob.glob(f"{cls_folder}/*.jpg"):  # Adjust if images are not .jpg
        img = load_img(img_path, target_size=image_size)
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(label)

# Convert to numpy arrays
images = np.array(images) / 255.0  # Normalize pixel values
labels = np.array(labels)

# Split data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
# Build CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(classes), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.3671 - loss: 2.6861 - val_accuracy: 0.2364 - val_loss: 1.4528
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 810ms/step - accuracy: 0.2907 - loss: 1.3538 - val_accuracy: 0.3273 - val_loss: 1.3361
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5774 - loss: 1.1850 - val_accuracy: 0.5455 - val_loss: 1.2185
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.7814 - loss: 0.8854 - val_accuracy: 0.4182 - val_loss: 1.3638
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.8238 - loss: 0.6079 - val_accuracy: 0.5636 - val_loss: 1.2634
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 881ms/step - accuracy: 0.9207 - loss: 0.4157 - val_accuracy: 0.4909 - val_loss: 1.3679
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9701 - loss: 0.2555 - val_accuracy: 0.6182 - val_loss: 1.5546
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9718 - loss: 0.1398 - val_accuracy: 0.5636 - val_loss: 1.6793
Epoc

In [ ]:
# Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 145ms/step - accuracy: 0.6386 - loss: 0.9867
Test Accuracy: 65.22%


In [ ]:
# Extract HOG features
X_train_hog = [hog(img.astype(np.float32), pixels_per_cell=(8, 8), cells_per_block=(2, 2), channel_axis=-1) for img in X_train]
X_test_hog = [hog(img.astype(np.float32), pixels_per_cell=(8, 8), cells_per_block=(2, 2), channel_axis=-1) for img in X_test]

# Train SVM
clf = SVC(kernel='linear')
clf.fit(X_train_hog, y_train)

# Evaluate
y_pred = clf.predict(X_test_hog)
print(classification_report(y_test, y_pred, target_names=classes))

              precision    recall  f1-score   support

  motorcycle       0.89      0.94      0.92        18
       Truck       0.64      0.74      0.68        19
         Car       0.81      0.87      0.84        15
         Bus       0.83      0.59      0.69        17

    accuracy                           0.78        69
   macro avg       0.79      0.78      0.78        69
weighted avg       0.79      0.78      0.78        69



In [ ]:
model.save("vehicle_classifier.h5")
